# Exploring the state of global primary education

*Reema George Dass 12144026, Simon Glatzer 12019265, Harald Hubinger 09426524, Valentin Schweitzer 51829840*

## Introduction

This analysis aims to provide an insight into the current state of global primary education by looking specifically at the primary school completion rate. In this case "current" means the last full year unaffected by the CoViD-19 pandemic (i.e. 2019), because taking into account the implications of the disruptions caused by it would go beyond the scope of this project and data availability drastically decreases from 2020 onwards. In the course of this analysis we will attempt to answer the following four questions:

- **Question 1** In all countries across the world, how many children finish primary school in 2019?
- **Question 2** How is the result from question 1 affected by the income of the countries?
- **Question 3** Is it possible to identify objectively reasons for children not finishing school. What factors influencing the completion rate can be identified?
- **Question 4** How well can predictions be made for the figures 5 and 10 years in the future?

### Data sources
#### Primary school completion rates.
The data we are working on is mainly taken from the World Bank databank, which compiles data accumulated by the UNESCO Institute for Statistics (UIS). However, additional sources had to be consulted in order to fill in missing values. These are data on primary education from the education ministry of the People's Republic of China, numbers from Statista on Nigeria, from MICS (Multiple Indicator Cluster Survey, a UNICEF programme) on Vietnam, from MICS-EAGLE (MICS specifically aimed at education) on Bangladesh, from UNICEF on Myanmar and from Kenya's ministry of education.  We tried to stick as closely as possible to data from official international institutions like the World Bank or UNESCO, because these are usually more transparent than specific governments, who might have an agenda behind their published data, which is not to say the data from such organisations can be expected to be 100% correct or unbiased. This is because the international organisations often also have to draw their information from governmental data, and we can think about different motivations of countries reporting data biased in one or the other direction. There could be incentives to report better than actural rates in order to polish a country's reputation. On the other hand reporting lower rates could be driven by availability of development funds. Further investigations into this would however go beyond the scope of this report.

The primary dataset treats "completion" as enrollment in the last grade of primary education. This, as stated in the original definition, does not consider students dropping out in the last grade. The rate of completion is calculated by dividing the number of enrollments by the number of people at the foreseen age of enrollment for the last grade. This sometimes results in values above 100%, which is caused by older or younger students finishing primary school For more information on this definition, consult the UNESCO Institute of Statistics website: http://www.uis.unesco.org/Education/".

Other countries may define completion rate differently in their official reports. Examples include:
* Graduation of last grade as a condition for "completion" (MICS, MICS-EAGLE, China, Kenya).
* Division by first-year enrollments as opposed to eligible population (China, Kenya).
* Percentage of children above a certain age who graduated from the last grade (MICS, MICS-EAGLE).
* A different definition of "primary school", i.e. possible incompatibilities to ISCED-1 (https://de.wikipedia.org/wiki/International_Standard_Classification_of_Education) Difficult to determine).

More detailed documentation on the compilation of the data is under `./datasets/combined_sources.md` and `./datasets/README.md`.

#### World population data
We used historic to present world population data from the World Bank available under https://data.worldbank.org/indicator/SP.POP.TOTL as https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv.

#### Human Development Index data
We extensively used data from the United Nations Development Human Development Reports (https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv) which was used there to compute the Human Development Index. This dataset, besides the HDI also contains all the variables contributing to the HDI as documented in https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_metadata.xlsx. The individual variables are documented under question 2 and 3 and as documented in question 3 we did not use the HDI directly but only its input variables as we would otherwise have counfounded the analysis for two of our main questions.

Especially this data also contains the GNI per capita on purchasing power parity for 2017 back to 1990. For financial data we  also investigated the GNIPC data from World Bank (https://data.worldbank.org/indicator/SE.PRM.CMPT.ZS), but chose the HDI data in the end.

#### Rurality data
We used percentages of rural population per country from World Bank Data available under https://data.worldbank.org/indicator/SP.RUR.TOTL.ZS as https://api.worldbank.org/v2/en/indicator/SP.RUR.TOTL.ZS?downloadformat=csv.

#### Child labour data
Child labour data has been taken from UNICEF available under https://data.unicef.org/topic/child-protection/child-labour/ as  https://data.unicef.org/wp-content/uploads/2019/10/XLS_Child-labour-database_May-2022.xlsx. This data is only available for a small set of countries, as child labour mainly plays a role for the very poor countries. UNICEF provides further information on methodology.

### Data quality and preprocessing

#### Overall data quality
Parts of this have already been treated under data sources.

For the school completion rates it can be said that data quality is different for different countries. We found fluctuations for example in the school completion rates of China which have to be attributed to the quality or methodology of measurement there. Nevertheless it's not really feasible to find good basis to decide if values should be excluded from the analysis. Under question 1 we also mention completion rates above 100% (especially in some small countries) for which explanations could be found. To further investigate their plausibility would also exceed what could be done for this report.

HDI, rurality and child labour data looked good overall.

#### Data cleaning
The collected data sets for school completion rates were manually cleaned by removing elements unnecessary for the analysis and correcting formatting errors in the files. Afterwards, the data sets were partially merged to create a more centralised data structure and make them easier to access. Most of the data was already in an easily accessible form as it was taken for example from the World Bank databank, which provides the information in an easy-to-use way.

#### Missing values
Many countries do **not regularly** report their primary education stats and some of them **don't report at all**. In the first case, the missing data was obtained by linear interpolation where we had data before and after 2019 and if only older data was available we used constant extrapolation (i.e. the value for the closest year). We considered this an adequate imputation strategy at first hand, but also try to model these values in question 4.

For the second case, we used the aforementioned additional data sets obtained from a variety of sources which might come at the price of reduced direct comparability. Also, these values we model for question 4 to see the differences between the regression and the data from different sources.

More on missing values can be found in the sections where it is relevant.

#### Outliers
Besides the aforementionned considerations on the "bumpyness" of some time series and the questionable "overachievers" we could not determine data from nuisance processes - also under the assumption that we're not able to put more scrutiny into the redaction of this data than UN / World Bank statistics which we consider very reliable data sources - and therefore did not exclude any outlying data.  

In [ ]:
# First, we will load all the data we need and import all the necessary packages to perform the various steps in the analysis. 
import pandas as pd
import json
import os
import pickle
import re
import geopandas as gp
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
import warnings
import pycountry

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectPercentile,SelectKBest,chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
%%capture --no-stderr
# We run some auxiliary notebooks to preprocess and merge data
wd = os.getcwd()
os.chdir("src/")
%run inter_extrapolate.ipynb
%run child_labour.ipynb
%run hdr.ipynb
%run rural.ipynb
%run uis_unesco.ipynb
%run merge_data.ipynb
os.chdir(wd)

In [ ]:
# Load the generated data
with open(os.path.join('target', 'merged.pickle'), 'rb') as fp:
    data = pickle.load(fp)
metadata = pd.read_csv(os.path.join('datasets', 'country_metadata_reduced.csv'), sep=',', index_col=0).reset_index()
country_list = metadata['Code'].tolist()

Now we can start our analysis with the first question.

## Question 1: In all countries across the world today, how many children finish primary school?

We will attempt to answer this question accurately by trying to find the primary school completion rate in 2019 for as many countries as possible based on the available numbers and then calculate a weighted global average using the number of children in primary school in each country to avoid the rate being dominated by a few large countries like China or India.

First, take the data from the merged data set containing all indicators. We are only interested in entries from 2019 and for which we have data concerning the primary completion rate, so we drop the rest.

In [ ]:
comp = data.loc[~data['ps_comp'].isna()]
temp = comp.loc[comp['year'] == 2019].reset_index(drop=True)

There are also aggregations of countries in this data frame, which are not relevant to our analysis.

In [ ]:
comp_2019 = temp[temp['iso3'].isin(country_list)]
comp_2019 = comp_2019[['iso3', 'country', 'pop', 'ps_comp', 'school_age_prim','psr_comp_source']]
display(comp_2019.sort_values(by='country'))

Now we have all the important data to answer our question. The result can be visualised by plotting it over a world map. One can easily observe, that the majority of the lower completion rates is centered arount sub-Saharan Africa. The colour of the borders shows how the data for this specific country was obtained.

In [ ]:
world = gp.read_file(gp.datasets.get_path("naturalearth_lowres")).rename(
    columns={"iso_a3": "iso3"}
)
world_comp = pd.merge(world, comp_2019, on="iso3", how="left")
world_comp["psr_comp_source"].replace({0.0: "Original Dataset", 1.0: "Auxilliary Dataset", 2.0: "Interpolated"}, inplace=True)

In [ ]:
w_ax = world_comp.plot(
    column="ps_comp",
    legend=True,
    legend_kwds={"label": "Completion Rate", "orientation": "horizontal", "pad": -0.015},
    figsize=(12,8),
    cmap="plasma"
)

w_ax.set_axis_off()
w_ax.set_title("Primary Education Completion Rate 2019")

colors = [(0,0,0), (255,255,255)]

w_ax_outline = world_comp[world_comp["psr_comp_source"] != "Original Dataset"].plot(
    column="psr_comp_source",
    legend=True,
    figsize=(12,8),
    facecolor="None",
    ax=w_ax,
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "None",
        #"hatch": "///",
        "label": "No Data",
    },
    linewidth=1,
    cmap="bwr",
)

outline_legend = w_ax_outline.get_legend()
for h in outline_legend.legendHandles[:-1]:
    h.set_linestyle("-")
    h.set_color(h.get_markerfacecolor())
    h.set_marker("")

Using the collected data, we can calculate a worldwide weighted average using the number of children in the relevant age group. This means we are going to drop a few entries in our list, which do not contain data for our weights. We will see however, that the result thus obtained is still meaningful.

In [ ]:
def mean_weighted(df, values, weights):
    mean = (df[values] * df[weights]).sum() / df[weights].sum()
    return mean

In [ ]:
comp_rate = 'ps_comp'
numb_children = 'school_age_prim'
pop = comp_2019['pop'].sum() / 1e9
print(f'The weighted global average completion rate in primary school, represented by a diverse range of countries with a total population of {pop:.2f} billion people, is {mean_weighted(comp_2019.dropna(), comp_rate, numb_children):.2f}%.')


Even though the final number we come up with, does not represent all countries in the world, it is nonetheless a very good estimate considering the composition and the overall population of the entities used in the analysis. The world's population in 2019 was roughly 7.76 billion people (https://de.statista.com/statistik/daten/studie/1694/umfrage/entwicklung-der-weltbevoelkerungszahl/) so we miss roughly 100 million people.

### Distribution of the primary school completion rates

In [ ]:
comp_2019.hist(column='ps_comp', bins=20)
plt.title("Histogram of Primary School Completion Rates")
plt.ylabel("Number of Countries")
plt.xlabel("Primary School Completion Rate")
plt.show()

As mentioned before completion rates above $100\%$ are artifacts of the measurement methods in place and can more or less be accounted to the $100 \%$ range. We observe a distribution which is strongly skewed to the left which is of course good, as many countries can provide primary education for the majority of their children. This observation is also interesting in conjunction with the questions to follow. 

## Question 2: How is the result from question 1 affected by the income of the countries?

Already by looking at the map drawn for Question 1 we can state the unsurprising hypothesis, that there should be a strong influence of economic prosperity, i.e. income, on primary school completion and therefore chose to explore this relationship. We decided to measure income by the Gross National Income Per Capita on Purchasing Power Parity for 2017 taken from the United Nations Human Development Reports data.

### Distribution of Incomes

In [ ]:
comp_income_2019 = temp[temp['iso3'].isin(country_list)]
comp_income_2019 = comp_income_2019[['iso3', 'country', 'pop', 'gnipc', 'ps_comp']]
# We have 10 NAs in gnipc. Apart from North Korea (pop ~ 2.5M) these are all "micro-countries". We can safely drop them.
comp_income_2019 = comp_income_2019.dropna()
comp_income_2019.hist(column='gnipc', bins=20)
plt.title("Histogram of Gross National Income Per Capita")
plt.ylabel("Number of Countries")
plt.xlabel("GNIPC PPP 2017 (USD)")
plt.show()

For the GNIPC we see a distribution which is even more extremely skewed, but to the right. Most countries have a GNI in the very low ranges compared to the very prospering countries. This could be expected, since income data is usually pareto distributed, but we don't go into further testing here.

### Income vs Primary School Completion
To account especially for the skewness of the income distribution we will take the log (base 10 for easy interpretability) of the GNIPC.

In [ ]:
comp_income_2019['gnipc_log'] = np.log10(comp_income_2019['gnipc'])
fig, ax = plt.subplots(figsize = (7, 7))
sns.regplot(
    data=comp_income_2019,
    x='gnipc_log',
    y='ps_comp',
    robust=True,
    line_kws = {"color": "red"}
)
#ax.set(xscale="log")
plt.title("GNIPC vs Primary School Completion with Robust Regression Line")
plt.ylabel("Primary School Completion Rate (%)")
plt.xlabel("log10 of GNIPC PPP 2017 (USD)")
plt.show()

We observe a strong trend between the logged GNIPC and the primary school completion rate. There seem to be some outlying observations, so we used robust regression. Nevertheless, the trend seems not best described by a linear model.

In [ ]:
fig, ax = plt.subplots(figsize = (7, 7))
sns.regplot(x = 'gnipc_log',
           y = 'ps_comp',
           data = comp_income_2019,
            lowess = True,
            line_kws = {"color": "red"}
           )
plt.title("GNIPC vs Primary School Completion with LOWESS Regression Line")
plt.ylabel("Primary School Completion Rate (%)")
plt.xlabel("log10 of GNIPC PPP 2017 (USD)")
plt.show()

The LOWESS regression line fits the data much better, and we see a "knee" in this plot. that low rates in primary schooling is an issue

In [ ]:
r, p = scipy.stats.pearsonr(comp_income_2019.ps_comp,comp_income_2019.gnipc_log)
print(f"We can calculate a pearson correlation for the log of the GNI per capita and primary school completion of {r} with a p-value of {p} and keep in mind that this only shows us the linear component of the correlation.")

### Relationship between primary school completion and GNI per capita over time

To illustrate that a relationship between primary school completion and incomes also holds over time we show temporal data from 1990 up to today of some selected countries (selected for low- to mid-income range and availability of data, therefore biased and just illustrative; missing data interpolated; nota bene though that Yemen in this plot shows a decrease in GNI while improving in primary school completion rates).

In [ ]:
sns.lineplot(data=comp.loc[comp.iso3.isin(["IND","IDN", "ETH", "BGD", "COD","KEN","MMR","IRN","YEM"])], x="year",y="gnipc",hue="iso3")
plt.title("GNIPC of Selected Countries Over Time")
plt.xlabel("Year")
plt.ylabel("GNI per Capita")
plt.show()

sns.lineplot(data=comp.loc[(comp.iso3.isin(["IND","IDN", "ETH", "BGD","COD", "KEN","MMR","IRN","YEM"])) & (comp.year>=1990)], x="year",y="ps_comp",hue="iso3")
plt.title("Primary School Completion Rate of Selected Countries Over Time")
plt.xlabel("Year")
plt.ylabel("Primary School Completion Rate")
plt.show()

### Conclusions

The plots show us that at a GNI per capita of slightly above USD $10\,000$ countries seem to reach the ability to cater for a more or less saturated degree of primary school completion. Beyond that threshold we can still see some  decrease in variability. Below this number variability seems higher and extreme poverty seems to be a clear hindrance for achievement. The level actually reached most probably also depend on other factors such as good governance or political priorities. We will go into further investiagions in question $3$.

There's of course a good basis for the assumption that the observed trend is not only a correlation in the data but is also based on causality as education costs a tremendous amount of money, it needs infrastructure and last but not least also cooperation from the families of the children, since under conditions of extreme poverty children are often engaged in child labour, either employed or in subsistence economy.

# Exploration of factors influencing the primary school completion rate (Q3)

First, we will load our main data set containing all the information about the different factors we have gathered. Before we start the analysis we drop several columns, which do not contain any relevant information and would just obscure our view of the results. The different indicators we are going to take a closer look at later are:
* hdi (Human Development Index)
* gii (Gender Inequality Index)
* gnipc (Gross National Income Per Capita)
* le (Life Expectancy)
* eys (Expected Years of Schooling)
* mys (Mean Years of Schooling)
* coef_ineq (Coefficient of human inequality)
* rural (percentage of population living in rural areas)
* compulsory (length of compulsory education in years)
* illiterate_adult (number of illiterate adults in country)
* illiterate_youth (number of illiterate youths in country)
* gov_exp (government expenditure on education as percentage of total government expenditure)
* child_labour (the rate of children who have to work)

From these indicators we will then try to derive reasons for children not finishing primary school. These (potentially) influencing factors cover a wide range of sectors including economics, health and education and as such will accurately represent what might positively or negatively affect the primary school completion rate of a country.

In [ ]:
#with open(os.path.join('target', 'merged.pickle'),'rb') as fp:
#    data = pickle.load(fp)

In [ ]:
df = data.copy()
df.shape

### Initial analysis

Before we start the analysis we will add another indicator composed of the government expenditure percentage, the population, the GNI and the number of children of primary school age to calculate a number that will in the end represent how much money a country spends for each student of primary school age. This is a better economic indicator than the plain percentage of government expenditure, because this takes both the size of the population and the total government expenditure into account.

In [ ]:
gov_exp_pc = df['gov_exp'] * (1/100) * df['pop'] * df['gnipc'] / df['school_age_prim']
df.insert(11, 'gov_exp_pc', gov_exp_pc)

Now that we have our full dataset we can now start exploring the data. First, we plot a visualization of the correlation matrix to get an initial view of the interrelations between the different indicators. Nevertheless, it would be better to get a more specific angle on the data regarding the primary school completion rate, since we are mostly interested in this metric.

In [ ]:
sns.heatmap(
    df.corr(numeric_only=True),
    cmap="coolwarm",
    annot=True,
    annot_kws={"fontsize":7}
)
plt.title("Heatmap of correlations")
plt.show()

To get this more specific view, we just look at the correlation of the completion rate and the other indicators. We can observe strong positive correlations with the HDI, the life expectancy and mean and expected years of schooling, while the completion rate shows a strong negative correlation with the GII and the overall coefficient of inequality.

In [ ]:
df[~df['child_labour'].isna()]

In [ ]:
for col in df.columns[[i for i in range(8, 24)]]:
    cor = df['ps_comp'].corr(df[col])
    print(f'The correlation between primary school completion rate and {col} is: {cor}')

The HDI (Human Development Index) will not be part of the further analysis, because it is defined using metrics we are already looking at specifically using other indicators, for example the life expectancy already covers the health aspect of the HDI in more detail. This way we avoid using data twice.

### Specific indicator analysis

We are now going to take a more detailed look at the aforementioned indicators, which we identified as the ones primarily affecting primary school completion rates (GNI is treated in question 2).

Firstly, we will analyse the positively correlated indicators and afterwords focus on the negatively correlated ones.

For now, we will take a look at the influence of the life expectancy on the completion rate. Here we can see a clear linear trend as with a higher life expectancy, meaning a better health infrastructure, also comes a higher completion rate in primary school.

In [ ]:
ax2 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'le']].dropna(), y='ps_comp', x='le', lowess = True, scatter_kws = {'alpha': .2}, line_kws = {"color": "red"})
ax2.set(ylabel='primary school completion rate in %', xlabel='life expectancy in years', title='scatter plot of life expectancy over primary school completion')
plt.show()

Next, we can observe a similarly clear trend for the expected years of schooling and the mean years of schooling. For the first one, we can see a plateau for higher values like for the GNI per capita, as after about 13 to 14 years of schooling there does not seem to be a discernible effect on the primary school completion rate anymore. A very similar case can be made for the mean years of schooling, where the same trend is observable with a saturation setting in after about 8 to 9 years.

In [ ]:
ax3 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'eys']].dropna(), y='ps_comp', x = 'eys', lowess = True, scatter_kws = {'alpha': .2}, line_kws = {"color": "red"})
ax3.set(ylabel='primary school completion rate in %', xlabel='expected years of schooling in years', title='scatter plot of expected years of schooling over primary school completion')
plt.show()

In [ ]:
ax4 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'mys']].dropna(), y='ps_comp', x = 'mys', lowess = True, scatter_kws = {'alpha': .2}, line_kws = {"color": "red"})
ax4.set(ylabel='primary school completion rate in %', xlabel='mean years of schooling in years', title='scatter plot of mean years of schooling over primary school completion')
plt.show()

The last two influencing factors correlate negatively with the primary school completion rate. First, we will take look at the Gender Inequality Index. A GII of 0 would mean absolute equality between both men and women, while a GII of 1 would mean absolute inequality between the genders. Here we observe a similar characteristic as we have seen before, with a linear trend in the beginning and at a GII of about 0.5 a plateau with no more improvement of the primary school completion rate.

In [ ]:
ax5 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'gii']].dropna(), y='ps_comp', x = 'gii', lowess = True, scatter_kws = {'alpha': .2}, line_kws = {"color": "red"})
ax5.set(ylabel='primary school completion rate in %', xlabel='GII (from 0 to 1)', title='scatter plot of Gender Inequality Index over primary school completion')
plt.show()

For the general coefficient of human inequality. A higher coefficient indicates a very high inequality in terms of education, health and income between the inhabitants of this country, while a lower coefficient means a more equal distribution of these. For this metric we have the same case, albeit a bit weaker than before, with the coefficient showing a higher completion rate as it decreases up until about 15 to 20, after which there is no observable effect on the primary school completion rate anymore.

In [ ]:
ax6 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'coef_ineq']].dropna(), y='ps_comp', x='coef_ineq', lowess = True, scatter_kws = {'alpha': .2}, line_kws = {"color": "red"})
ax6.set(ylabel='primary school completion rate in %', xlabel='coefficient of inequality (0 to 100)', title='scatter plot coefficient of inequality over primary school completion')
plt.show()

Since child labour was also mentioned as a possible factor influencing the primary school completion rate of a country, we are also going to take a quick look at this indicator as well. While we do have fewer data points for this metric, one can nonetheless see a clear correlation between a higher child labour rate and a lower primary school completion and vice versa. It should however be noted, that child labour data is mostly a problem in less developed countries.

In [ ]:
ax7 = sns.regplot(data=df[df.psr_comp_source != 2][['ps_comp', 'child_labour']].dropna(), y='ps_comp', x='child_labour',line_kws = {"color": "red"})
ax7.set(ylabel='primary school completion rate in %', xlabel='child labour rate in %', title='scatter plot child labour rate over primary school completion')
plt.show()

### Question 3 - Conclusion

After analysing all the relevant influencing factors, we can now try to identify reasons for children not finishing primary school.

The first major factor we looked at was the life expectancy, which as an indicator hints to a healthy population supported by a strong health care system. From this one might gather that a healthy body of students is more likely to graduate than one affected by illnesses, which does seem to make sense, since being ill can negatively affect one's cognitive capabilites apart from the fact that an illness might even force children to stay home and not attend school.

Two other major points are the mean and expected years of schooling which in this case might stand to represent the development of the educational system in a country, a longer time in school meaning a more developed system with an increased focus on secondary and tertiary education. That focus might reduce the importance of a primary education alone, while strengthening it as the prerequisite for pursuing a higher education and as such becoming even more important.

Furhtermore, child labour was also identified as a factor affecting the graduation rate. This does also seem natural, since a child who has to participate in sustaining its family at an early age might not have the time to pursue an education.

At last, the inequality in a society was also identified as a reason for children not graduating from primary school. This includes both general inequality in society, meaning the individual's access to health care, wealth and education, regardless of their respective quality, and gender inequality, this being the difference between the genders male and female. This does seem to make sense, as a more unequal distribution of resources in a society allows fewer people the freedom to pursue an education, not plagued by any of the other factors mentioned before.

It is not really possible to "objectively" identify reasons for children not finishing primary school since this rate is influenced by so many vastly different factors, we can only establish hypotheses on what might alter the state of primary completion rate in a country.

## Question 4: How well can predictions be made for the figures 5 and 10 years in the future?

From the other questions we looked at in our analysis we see, that there's a strong dependence of educational achievement and other especially economy and development related variables. These variables are themselves very hard to predict for which reason for example forecasts of economic growth are usually only done for comparably short periods of time, e.g. for governmental budget planning. Disruptive events like the COVID-19 pandemic (which is not depicted in our data) also make clear how quickly even relatively short time forecasts can become obsolete and global trends can be interrupted. To this end it's very instructive to have a short glimpse at https://hdr.undp.org/data-center/human-development-index#/indicies/HDI and look at the turn from 2019 to 2020.

It seems that this strong correlation with external factors also doesn't allow for useful time series modeling which would allow to predict the completion rates based on their inner dynamics (as we could probably do for monthly rainfall). So any attempt to build a model for primary school completion into the longer future would more or less build on sandy ground.

For this reason we tried to use the insights gained from external factors influencing primary school completion to complete the picture for 2019 and estimate the rates for those countries where no direct data was available in our primary data set.

In [ ]:
warnings.filterwarnings("ignore")
#the dataset has values of psr_comp_source=0,1,2
primary_df = df.copy()
#making a unique list of country iso codes
country_list = primary_df['iso3'].unique()

In the following steps the data is cleaned and the missing values are being filled and finally train the model to be able to predict all the missing values we found in the main dataset.

In [ ]:
#Capture overall mean to fill in na values in the following steps
primary_df_mean_values = primary_df.mean(axis=0, numeric_only=True)

The meaning of psr_comp_source:

1.   psr_comp_source=0 means the data for primary school completion rate was from the main dataframe.
2.   psr_comp_source=1 means the data for primary school completion rate was from the different dataframe.
3.   psr_comp_source=0 means the data for primary school completion rate was derived by inter/exterpolation methods.

So we use the dataset with psr_comp_source=0 to train and test the model and the rest for prediction.

In [ ]:
my_valid_values=[0.0]
model_fitting_df=primary_df.loc[primary_df['psr_comp_source'].isin(my_valid_values) ]
my_valid_values=[1.0,2.0]
to_predict_df=primary_df.loc[primary_df['psr_comp_source'].isin(my_valid_values) ]

### The missing values are filled in two levels, as mentioned below:


1.   Capturing mean values coutry wise, while we try to replace the na values with this value in case of a missing value.

2.   Even after filling the na values based step1, there are possibilities to have na values in our dataset as the whole group based on countries could have the missing values yielding to na values, hence in this situation I am filling it with the average mean of that column irrespective of the country.

In [ ]:
primary_df=pd.DataFrame()
#performing step1 for filling missing data
for i in country_list:
    middle_df=model_fitting_df.loc[model_fitting_df['iso3'] == i]
    middle_df.fillna(middle_df.mean(numeric_only=True), inplace=True) 
    primary_df=primary_df.append(middle_df, ignore_index = True)

In [ ]:
#performing step2 for filling missing data
nan_cols = [i for i in primary_df.columns if primary_df[i].isnull().any()]
if len(nan_cols)>0:
    for i in nan_cols:
        primary_df[i].fillna(primary_df_mean_values[i], inplace=True)

Testing if any values are na/null and making sure the length and the shape of the dataframe stay same.

In [ ]:
nan_cols = [i for i in primary_df.columns if primary_df[i].isnull().all()]
len(primary_df.index)
primary_df.isnull().values.any()
primary_df[primary_df.isna().any(axis=1)]

For training the model the feautures shold be numeric, as we have a couple of columns like country name and iso3 i.e. the iso codes in alphabets to denote country name has to be removed but as we need it as a part of dataframe in order to find the relevance of predicted data wrt the country, so we convert this categorical variables as columns and add boolean values based on its relevance.

In [ ]:
countries_one_hot = pd.get_dummies(primary_df["iso3"], prefix="iso3")
primary_df.drop(["iso3"], inplace=True, axis=1)

Using the feature selection method "SelectPercentile" in order to select the important/most correalted features for the model training. 
Currently, the most optimal percentage was 50, as we are keeping it exactly at half and the selected features based on this percentage are taken in to consideration to train our model.

In [ ]:
x=primary_df[[ 'pop', 'year', 'gii', 'gnipc', 'gov_exp_pc', 'le', 'eys', 'mys',
       'coef_ineq', 'rural', 'compulsory', 'illiterate_adult',
       'illiterate_youth', 'school_age_prim', 'oo_school_prim', 'gov_exp'
       ]].copy()
y=primary_df[[ 'ps_comp']].copy()

feature_names = SelectPercentile(percentile=50).fit(x, y)

important__features=list(feature_names.get_feature_names_out())
important__features.append("ps_comp")

selected__features = list(important__features) # list() to create new list
selected__features.remove("ps_comp")

FS_task4_to_train_test=primary_df[important__features].copy()
FS_task4_to_train_test = pd.concat([FS_task4_to_train_test, countries_one_hot], axis=1)
print(f'The features selected are: {selected__features}')

Splitting the main dataset into train and test and validate if the model works fine, then we predict the actual needed values.
The linear model dint show the promising results hence the decision to choose a non-linear model was made i.e. Random forest regression model.

In [ ]:
train, test = train_test_split(FS_task4_to_train_test, test_size=0.1,random_state=26)
train_x=train.drop('ps_comp',axis=1)
train_y=train[[ 'ps_comp']]
test_x=test.drop('ps_comp',axis=1)
test_y=test[['ps_comp']]

Fitting the model with the selected model and predicting the test set.
We also tried GridSearchCV with various parameter grid combinations, but the performance didn't improve from the current state hence we are going back to a simpler version of the model fit.

In [ ]:
clf = RandomForestRegressor(random_state=26)

clf.fit(train_x, train_y)
predict = clf.predict(test_x)

y_pred=pd.DataFrame(predict, columns = ['ps_comp'])

The metrics of relevance for validating the model performance:

In [ ]:
y_true = test_y
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_true, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_true, y_pred))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_true, y_pred, squared=False))
print('Explained Variance Score:', metrics.explained_variance_score(y_true, y_pred))
print('R^2:', metrics.r2_score(y_true, y_pred))

Plotting the test values and predicted values it's very evident that they are linear, safe to assume that to some level the data predicted is accurate.

In [ ]:
sns.regplot(x=test_y['ps_comp'],y=y_pred['ps_comp'],scatter_kws = {'alpha': .2}, line_kws = {"color": "red"});
plt.xlabel('True values')
plt.ylabel('Predictions')
plt.show()

# Final prediction of all the data with missing completion rate.

The filling of the data is required as the model needs the data without null values, hence I am following the same method to fill the missing values like the train/train data was filled, to keep te data consistent.

In [ ]:
#Filling missing data.
final_prediction_df=pd.DataFrame()
for i in country_list:
    middle_df=to_predict_df.loc[to_predict_df['iso3'] == i]
    middle_df.fillna(middle_df.mean(numeric_only=True), inplace=True) 
    final_prediction_df=final_prediction_df.append(middle_df, ignore_index = True)

nan_cols = [i for i in final_prediction_df.columns if final_prediction_df[i].isnull().any()]
final_prediction_df[nan_cols].isnull().values.any()

if len(nan_cols)>0:
    for i in nan_cols:
        final_prediction_df[i].fillna(primary_df_mean_values[i], inplace=True)

to_predict_df

In [ ]:
final_prediction_df.isnull().values.any()
final_prediction_df[final_prediction_df.isna().any(axis=1)]
countries_one_hot = pd.get_dummies(final_prediction_df["iso3"], prefix="iso3")
final_prediction_df.drop(["iso3"], inplace=True, axis=1)

In [ ]:
# renaming the column as this could cause inconsistency, we are not going to refer this at all, as this wasnt the part of the main dataframe
final_prediction_df.rename(columns={"ps_comp": "manual_ps_comp"}, errors="raise")

#dropping the column as we have other reference wrt country.
final_prediction_df.drop(columns='country',inplace=True)

prediction_set=pd.concat([final_prediction_df, countries_one_hot], axis=1)

#creating a set of independent variables for prediction
prediction_set = prediction_set[train_x.keys()]

### Predicting values:

In [ ]:
predict_values = clf.predict(prediction_set)
predict_values=pd.DataFrame(predict_values, columns = ['ps_comp'])
predict_values

In [ ]:
# https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
def undummify(df, prefix_sep: str = "_", prefix_matches: str = "iso3"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns if item.split(prefix_sep)[0] == prefix_matches
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [ ]:
complete_predicted_df = pd.concat([prediction_set,predict_values], axis=1, join="inner")
iso_columns = [col for col in complete_predicted_df.columns if col.startswith("iso3_")]

long_format = complete_predicted_df.drop(columns=iso_columns).copy()
long_format["iso3"] = undummify(complete_predicted_df)["iso3"]


We integrate the data from our regression model 

In [ ]:
comp_2019_regr = comp_2019.copy()
regr_2019 = long_format[long_format.year==2019]


In [ ]:
comp_2019_regr.set_index('iso3', inplace=True)
comp_2019_regr.update(regr_2019.set_index('iso3'))
comp_2019_regr.reset_index(inplace=True)
comp_2019_regr["psr_comp_source"].replace({0.0: "Original Dataset", 1.0: "Modelled Values", 2.0: "Modelled Values"}, inplace=True)

In [ ]:
world = gp.read_file(gp.datasets.get_path('naturalearth_lowres'))
world_comp_2019_regr = pd.merge(world, comp_2019_regr, left_on="iso_a3", right_on="iso3", how="outer")
year = 2019

w_ax = world_comp_2019_regr.plot(
    column="ps_comp",
    legend=True,
    legend_kwds={"label": "Completion Rate", "orientation": "horizontal", "pad": -0.015},
    figsize=(12,8),
    cmap="plasma"
)

w_ax.set_axis_off()
w_ax.set_title("Primary Education Completion Rate 2019 with Regression")

colors = [(0,0,0), (255,255,255)]

w_ax_outline = world_comp_2019_regr[world_comp_2019_regr["psr_comp_source"] != "Original Dataset"].plot(
    column="psr_comp_source",
    legend=True,
    figsize=(12,8),
    facecolor="None",
    ax=w_ax,
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "None",
        #"hatch": "///",
        "label": "No Data",
    },
    linewidth=1,
    cmap="bwr",
)

outline_legend = w_ax_outline.get_legend()
for h in outline_legend.legendHandles[:-1]:
    h.set_linestyle("-")
    h.set_color(h.get_markerfacecolor())
    h.set_marker("")

In [ ]:
world_avg_comp = np.average(comp_2019_regr.dropna().ps_comp, weights=comp_2019_regr.dropna().school_age_prim)
world_avg_q1 = np.average(comp_2019.dropna().ps_comp, weights=comp_2019.dropna().school_age_prim)
print(f'The weighted global average completion rate in primary school, represented by a diverse range of countries with a total population of {pop:.2f} ' \
    f'billion people, with missing values estimated by our regression model, is {world_avg_comp:.2f}%.')
print(f'The weighted global average obtained in the first question using only inter-/extrapolation and auxiliary datsets was {mean_weighted(comp_2019.dropna(), comp_rate, numb_children):.2f}%.')
print(f'The estimate thereby increased by {world_avg_comp - world_avg_q1:.2f}% which can be considered substantial.')

## Conclusions
Accurate primary school completion rates are quite difficult to adequately determine. For some countries hardly any data can be found. We investigated a pluritude of datasets which differed in applied methodology for their respective generation. Data collection constituted a tremendous amount of the total effort for this project. The main challenge in the work was to keep track of the variety of data used and to use it in a sensible way.

We found many influential factors for the school completion rates and investigated only the most promising among them. Remarkably, there is a pattern in all the dependencies, from which one could hypothesize, that at some level of development of a country a saturated level for the primary school completion can be achieved.

We juxtaposed worldwide achieved completion rates found by using our main data set and linear interpolation / constant extapolation / auxiliary datasets with values we could find by using a regression model.

The regression model fitted quite well on the test data and seems to be useful to estimate completion rates for countries which don't report (regularly). Nevertheless, we do not consider such a model useful to predict future values, as a main part of its explanatory variables would have to be forecast themselves. Further to this the random forest regression model we used is not easily to interpret.

We found a significant difference between these two values which might be due to the fact that constant extrapolation does not actually account for global development. The temporal scope here ends in 2019, it would however be interesting to use the model to calculate current values and compare them to reported ones since the constant uprise in global development as can be seen in the overall development of the HDI changed to a slight decline in 2020 with the COVID pandemic.

## Division of work

### Reema George Dass
 - Worked with the team to find all the relevant data sets for the problem statement and the related data that could possibly influence it.
 - Worked on exploring the data and cleaning the data set relevant to Q2.
 - Also used various processes to fill in the missing data and excluded outliers for exploration.
 - Merged relevant data required for finding correlation between the income group and the completion rate and found partial relation, but an alternative data set had more historic data which showed a better trend, was used to answer Q2.
 - I worked on Q4, predicting the primary completion rate for missing countries.
 - Exploring the data set we derived after Q3, which was the input for Q4, cleaning and finding strategies to fill missing data.
 - Selecting the optimal set of features and finding a better fitting model.
 - Fitting the models and validating the best one, and predicting the values required with exploration plots.
 - Cleaning and documenting Q4 and integrating Q4 to the primary notebook.
 
### Simon Glatzer
 - Researched and collected data on enrolment and different indicators possibly affection completion rate
 - Manual cleaning of primary data set (including metadata) to prepare data for further use
 - Tested all csv-files to make sure they can be uniformly loaded into pandas and used for the project.
 - Created notebooks as introduction and to showcase different peculiarities of the data (which countries are missing, and weighted average of the completion rate omitting said missing countries).
 - Researched additional data to fill missing completion rate data
 - Introduction and Q1 on the final presentation notebook
 - Work on Q3 for the final presentation notebook
 - Processed child labour data and added it to Q3 in final presentation notebook
 - Created management summary
 - Work on presentation slides

### Harald Hubinger
 - Create project plan, finalize project breakdown for presentation Dec 13
 - Coordinate meetings
 - Minor manual cleaning of the rurality data set (World Bank) for processing. Prepare this data for further use.
 - Research on gender inequality data, fetch Gender Inqueality Index (GII) data together with Human Development Index, GNI per capita and other indexes 1990 - 2021 from UN Development
 - Extract GNI, HDI and GNI per capita data and prepare for further use.
 - Convert Democracy Index PDF to .xlsx for further investigation via ilovepdf.com, some merging tests (and then throw it away...)
 - prepare further auxiliary data (UNESCO finance data, population data, decompose hdi)
 - gather and prepare further auxiliary data (UNESCO government expenditure, illiteracy...)
 - gather child labour data for group working session (2023/01/05)
 - Exploratory Analysis, elaboration of question 2, redact some parts on question 1
 - Extend introduction and redact complete notebook, project cleanup, integration of auxiliary notebooks
 - Modify graphics in question 3
 - Merge and compare results from modelling with original data from question 1, compare with prior results, add world map with the new values, write final conclusions.

### Valentin Schweitzer
 - Collected datasets for primary school completion rate.
 - Determined large countries with missing data.
 - Researched sources for alternative completion rate data and filled in missing data for large countries where possible.
 - Documented alternative sources for completion rate data and their relations to the original data source (mostly differences in definitions).
 - Created simple inter/extrapolation for completion rate data along with data source indicator.
 - Created initial world map plots.
 - General plot formatting (accurate/legible legends).
 - Helped with creation and evaluation of regressor pipeline:
   - One-hot encoding and de-encoding of countries.
   - Ensuring consistency of random states.
   - Refactoring.
 - Some LaTeX formatting.
 - General git support.
